In [1]:
import sys, os
os.listdir()
sys.path.append('/usr/local/anaconda3/lib/python3.6/site-packages')
sys.path.append('/home/snuthakk/.local/bin')
import numpy as np
import pandas as pd

In [2]:
#To unzip .gz files
# import gzip
# import shutil
# with gzip.open('PROCEDURES_ICD.csv.gz', 'rb') as f_in:
#     with open('PROCEDURES_ICD.csv', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# Importing data

# Noteevents dataset

In [3]:
import pandas as pd
noteevents = pd.read_csv('NOTEEVENTS.csv', sep = ',')
print(len(noteevents))
noteevents[:10]

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2083180


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
5,179,53181,170490.0,2172-03-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-3-5**] D...
6,180,20646,134727.0,2112-12-10,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2112-12-8**] ...
7,181,42130,114236.0,2150-03-01,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2150-2-25**] ...
8,182,56174,163469.0,2118-08-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-8-10**] ...
9,183,56174,189681.0,2118-12-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-12-7**] ...


In [11]:
noteevents['TEXT'].nunique()

2023185

In [8]:
(1215197/2083180)*100

58.33374936395319

In [4]:
# noteevents[:10].to_csv('noteevents_tmp1.csv')

In [5]:
pd.set_option('display.max_colwidth',1)
noteevents1 = noteevents[['SUBJECT_ID', 'HADM_ID', 'TEXT']]
noteevents1[:10]

SUBJECT_ID   HADM_ID  \
0  22532       167853.0   
1  13702       107527.0   
2  13702       167118.0   
3  13702       196489.0   
4  26880       135453.0   
5  53181       170490.0   
6  20646       134727.0   
7  42130       114236.0   
8  56174       163469.0   
9  56174       189681.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [6]:
len(noteevents1)

2083180

In [18]:
noteevents1 = noteevents1.to_csv('noteevents@3columns.csv', index=False)

In [4]:
noteevents1 = pd.read_csv('noteevents@3columns.csv')
len(noteevents1)

2083180

In [49]:
print(noteevents1['SUBJECT_ID'].nunique())
print(noteevents1['HADM_ID'].nunique())

46146
58361


In [48]:
#count unique records by two columns 'SUBJECT_ID', 'HADM_ID'
noteevents1.groupby(['SUBJECT_ID', 'HADM_ID']).ngroups

58361

### Joining procedures@seq1 dataset with noteevents dataset

In [25]:
#converting pandas dataframe 'noteevents1' to dask dataframe
import dask.dataframe as dd
# ddf = dd.read_csv('noteevents@3columns.csv', sep=',')
ddf = dd.from_pandas(noteevents1, npartitions=1)
ddf.head(6)

,SUBJECT_ID,HADM_ID,TEXT
0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,Admission Date: [**2162-3-3**] D...
5,53181,170490.0,Admission Date: [**2172-3-5**] D...


In [26]:
### Joining diagnosis_seq1 dataset with noteevents dataset

In [12]:
import dask.multiprocessing
import csv
print("1")
# Read in the csv files.
df1 = dd.read_csv('procedures@seq1.csv')
print("2")
# Merge the csv files.
df = dd.merge(df1, ddf, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print("3")
# # Write the output.
df.head()

1
2
3


,SUBJECT_ID,HADM_ID,ICD9_CODE,TEXT
0,2592,130856,9671,Admission Date: [**2183-6-5**] Discharg...
1,2592,130856,9671,PATIENT/TEST INFORMATION:\nIndication: Left ve...
2,2592,130856,9671,Atrial fibrillation with a controlled ventricu...
3,2592,130856,9671,Atrial fibrillation with a moderate ventricula...
4,2592,130856,9671,Atrial fibrillation with a controlled ventricu...


In [13]:
len(df)

1769301

In [14]:
import time
start_time = time.time()
df.to_csv('proce-*.csv', index=False)
print("--- %s seconds ---" % (time.time() - start_time))

--- 226.1217861175537 seconds ---


In [27]:
tmp_proce = pd.read_csv('proce-0.csv')
len(tmp_proce)

1769301

In [28]:
tmp_proce['ICD9_CODE'].nunique()

1274

In [30]:
proce_top10 = tmp_proce[tmp_proce['ICD9_CODE'].isin([9671, 9955, 3893, 9604,  640, 9672, 3521, 3615, 3612,   66])]
len(proce_top10)

632994

In [33]:
proce_top10 = proce_top10[['ICD9_CODE', 'TEXT']]
proce_top10['ICD9_CODE'].unique()

array([9671, 9672, 3612, 3615, 3893, 3521, 9604, 9955,  640,   66])

In [35]:
proce_top10['TEXT'].nunique()

618063

In [36]:
proce_top10.to_csv('proce_top10.csv', index=False)

In [37]:
proce_top50 = tmp_proce[tmp_proce['ICD9_CODE'].isin([9671, 9955, 3893, 9604,  640, 9672, 3521, 3615, 3612,   66, 9390,
        311, 4513, 3613, 3601, 3995, 9983,  966,  159, 3522, 9904,  331,
        131, 9915, 3950, 3512, 3611, 3761, 3972, 5491, 4443, 3723,  387,
       3614, 3734, 3897, 3891, 5059, 8841, 3523, 8622, 7935, 3979, 3491,
       3722, 9910, 4516,   22, 3929, 4562])]
len(proce_top50)

1215197

In [38]:
proce_top50 = proce_top50[['ICD9_CODE', 'TEXT']]
proce_top50['ICD9_CODE'].nunique()

50

In [39]:
proce_top50['TEXT'].nunique()

1184586

In [40]:
proce_top50.to_csv('proce_top50.csv', index=False)

### Joining diagnosis_seq1 dataset with noteevents dataset

In [36]:
import dask.multiprocessing
import csv
print("1")
# Read in the csv files.
df1 = dd.read_csv('diagnosis@seq1.csv')
print("2")
# Merge the csv files.
df = dd.merge(df1, ddf, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print("3")
# # Write the output.
df.head()

1
2
3


SUBJECT_ID  HADM_ID ICD9_CODE  \
0  109         172335   40301      
1  109         172335   40301      
2  109         172335   40301      
3  109         172335   40301      
4  109         172335   40301      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [37]:
len(df)

1851243

In [38]:
import time
start_time = time.time()
df.to_csv('diag-*.csv', index=False)
print("--- %s seconds ---" % (time.time() - start_time))

--- 223.52015924453735 seconds ---


In [3]:
tmp_diag = pd.read_csv('diag-0.csv')
len(tmp_diag)

1851243

In [4]:
tmp_diag.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE,TEXT
0,109,172335,40301,Admission Date: [**2141-9-18**] ...
1,109,172335,40301,PATIENT/TEST INFORMATION:\nIndication: Pericar...
2,109,172335,40301,Sinus rhythm\nRightward axis\nSince previous t...
3,109,172335,40301,"Chief Complaint: hypotension, altered mental ..."
4,109,172335,40301,"Chief Complaint: hypotension, altered mental ..."


In [8]:
tmp_diag['ICD9_CODE'].nunique()

2769

In [85]:
diag_top10 = tmp_diag[tmp_diag['ICD9_CODE'].isin(['V3000', '41401', 'V3001', '0389', '41071', '4241', '51881', '431',
       'V3101', '486'])]
len(diag_top10)

677738

In [87]:
diag_top10 = diag_top10[['ICD9_CODE', 'TEXT']]
diag_top10['ICD9_CODE'].unique()

array(['41401', 'V3001', 'V3000', 'V3101', '41071', '431', '4241', '0389',
       '51881', '486'], dtype=object)

In [89]:
diag_top10['TEXT'].nunique()

662858

In [88]:
diag_top10.to_csv('diag_top10.csv', index=False)

In [7]:
diag_top10 = pd.read_csv('diag_top10.csv')
len(diag_top10)

677738

In [8]:
diag_top10['ICD9_CODE'].nunique()

10

In [91]:
diag_top50 = tmp_diag[tmp_diag['ICD9_CODE'].isin(['V3000', '41401', 'V3001', '0389', '41071', '4241', '51881', '431',
       'V3101', '486', '5070', '4280', '4240', '430', '5849', '41011',
       '41041', '5789', '5770', '1983', '41519', '43411', '43491',
       '42731', '99859', '03842', '85221', '25013', '4373', '42823',
       '99662', '56212', '42833', '49121', '4271', '5712', '51884',
       '29181', 'V3401', '4321', '43310', '99811', '03849', '85220',
       '03811', '042', '4414', '1623', '0380', '5715'])]
len(diag_top50)

1058988

In [92]:
diag_top50 = diag_top50[['ICD9_CODE', 'TEXT']]
diag_top50['ICD9_CODE'].unique()

array(['41401', 'V3001', '5715', 'V3000', '41011', 'V3101', '99859',
       '41071', '430', '431', '042', '4241', '43310', '0389', '4373',
       '51881', '1983', '5070', '41041', 'V3401', '5712', '4280', '5789',
       '99811', '0380', '4321', '85220', '4240', '56212', '85221',
       '99662', '51884', '42823', '1623', '486', '03842', '4414', '03811',
       '5770', '41519', '25013', '43411', '5849', '03849', '42833',
       '43491', '42731', '49121', '4271', '29181'], dtype=object)

In [93]:
diag_top50['TEXT'].nunique()

1033147

In [94]:
diag_top50.to_csv('diag_top50.csv', index=False)

### checking unique texts in proce_top10 and diag_top10

In [111]:
a = diag_top10['TEXT'].isin(proce_top10['TEXT'])
a.value_counts()

True     385466
False    292272
Name: TEXT, dtype: int64

In [9]:
from collections import Counter
print(Counter(tmp_diag['ICD9_CODE']))

Counter({'V3001': 157174, 'V3101': 109767, 'V3000': 87594, '0389': 76775, '41401': 64187, '51881': 57269, '41071': 50747, 'V3401': 30819, '431': 28037, '4241': 25728, '430': 21733, '486': 20460, '5070': 19453, '5770': 19421, '51884': 17387, '4280': 16334, 'V3100': 13048, '5849': 12764, '99859': 12043, '41011': 11993, '4240': 11947, '03811': 11273, '41041': 10749, '03842': 9993, '03849': 9614, '0380': 9602, '5712': 8932, '42823': 8730, '41519': 8468, '43411': 8213, '99662': 8187, '5789': 8111, '44101': 8096, '42833': 8089, '570': 7947, '042': 7737, '43491': 7633, '42731': 7553, '85221': 7157, '4271': 7031, '0388': 6737, '49121': 6670, '4210': 6489, '4414': 6401, '1983': 5966, '20500': 5775, '4321': 5488, '5570': 5403, '5722': 5331, '1623': 5224, '00845': 4862, '3962': 4825, '5715': 4778, '1628': 4776, '85220': 4756, '4412': 4615, '56212': 4610, '99661': 4565, '44024': 4473, '56081': 4292, '0382': 4082, '25013': 4059, '42843': 4047, '03812': 4018, '5845': 4015, '9974': 4009, '41001': 397

### End of creating temporary diagnosis and noteevents only datasets

In [48]:
# noteevents1['SUBJECT_ID'][:100]

In [17]:
# noteevents1['TEXT_stripped'] = noteevents1['TEXT'].str.split('Service').str[1]

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
#check no of na's in all columns
# noteevents1.isna().sum()

SUBJECT_ID       0      
HADM_ID          231836 
TEXT             0      
TEXT_stripped    2020163
dtype: int64

In [19]:
noteevents1['TEXT_stripped_service'] = noteevents1['TEXT'].str.split('service').str[1]
noteevents1.isna().sum()

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


SUBJECT_ID               0      
HADM_ID                  231836 
TEXT                     0      
TEXT_stripped            2020163
TEXT_stripped_service    1996593
dtype: int64

In [20]:
noteevents1['TEXT_stripped_chiefcomp'] = noteevents1['TEXT'].str.split('Chief Complaint').str[1]
noteevents1.isna().sum()

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


SUBJECT_ID                 0      
HADM_ID                    231836 
TEXT                       0      
TEXT_stripped              2020163
TEXT_stripped_service      1996593
TEXT_stripped_chiefcomp    1939132
dtype: int64

In [21]:
noteevents1['TEXT_stripped_history'] = noteevents1['TEXT'].str.split('HISTORY OF PRESENT ILLNESS').str[1]
noteevents1.isna().sum()

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


SUBJECT_ID                 0      
HADM_ID                    231836 
TEXT                       0      
TEXT_stripped              2020163
TEXT_stripped_service      1996593
TEXT_stripped_chiefcomp    1939132
TEXT_stripped_history      2070650
dtype: int64

In [122]:
#convert all text in noteevents1['TEXT'] to lower case
noteevents1['TEXT'] = noteevents1['TEXT'].str.lower()

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [123]:
def index_default(i, char):
    """Returns the index of a character in a line, or the length of the string
    if the character does not appear.
    """
    try:
        retval = i.index(char)
    except ValueError:
        retval = 100000
    return retval

s = []
def split_log_line(i):
    """Splits a line at either a period or a colon, depending on which appears 
    first in the line.
    """
    if index_default(i, "chief complaint") < index_default(i, "history of present illness"):
        a = i.split('chief complaint')[1]
        b = 'Chief complaint' + a
        s.append(b)
    elif index_default(i, "history of present illness") < 100000:
        a = i.split('history of present illness')[1]
        b = 'History of present illness' + a
        s.append(b)
    else:
        s.append('')

In [124]:
for i in noteevents1['TEXT']:
    split_log_line(i)

In [125]:
noteevents1['TEXT_required'] = s

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
noteevents1[:10]

SUBJECT_ID   HADM_ID  \
0  22532       167853.0   
1  13702       107527.0   
2  13702       167118.0   
3  13702       196489.0   
4  26880       135453.0   
5  53181       170490.0   
6  20646       134727.0   
7  42130       114236.0   
8  56174       163469.0   
9  56174       189681.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [134]:
#Replacing all blank value with NaN
# noteevents2 = noteevents1['TEXT_required'].replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [133]:
noteevents1.isna().sum()

SUBJECT_ID       0      
HADM_ID          231836 
TEXT             0      
TEXT_required    1892497
dtype: int64

In [138]:
#drop rows with nan values in 'TEXT_required' column
noteevents2 = noteevents1.dropna(subset=['HADM_ID', 'TEXT_required'])
noteevents2 = noteevents2[['SUBJECT_ID', 'HADM_ID', 'TEXT_required']]
len(noteevents2)

189101

In [139]:
noteevents2.to_csv('noteevents2.csv')

In [50]:
noteevents_190k = pd.read_csv('noteevents2.csv')
noteevents_190k.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,TEXT_required
0,1,13702,107527.0,History of present illness: this is an 81-yea...
1,2,13702,167118.0,"Chief complaint:\n81 yo f smoker w/ copd, seve..."
2,3,13702,196489.0,Chief complaint:\ncopd exacerbation/shortness ...
3,4,26880,135453.0,Chief complaint:\nmr. [**known lastname 1829**...
4,5,53181,170490.0,Chief complaint:\nmeningioma\n\nmajor surgical...


In [52]:
print(len(noteevents_190k))

189101


In [54]:
noteevents_190k = noteevents_190k[['SUBJECT_ID', 'HADM_ID', 'TEXT_required']]

In [55]:
#converting pandas dataframe 'noteevents1' to dask dataframe
import dask.dataframe as dd
# ddf = dd.read_csv('noteevents@3columns.csv', sep=',')
ddf = dd.from_pandas(noteevents_190k, npartitions=1)
ddf.head(6)

,SUBJECT_ID,HADM_ID,TEXT_required
0,13702,107527.0,History of present illness: this is an 81-yea...
1,13702,167118.0,"Chief complaint:\n81 yo f smoker w/ copd, seve..."
2,13702,196489.0,Chief complaint:\ncopd exacerbation/shortness ...
3,26880,135453.0,Chief complaint:\nmr. [**known lastname 1829**...
4,53181,170490.0,Chief complaint:\nmeningioma\n\nmajor surgical...
5,20646,134727.0,Chief complaint:\nhypoxia\n\nmajor surgical or...


In [56]:
import dask.multiprocessing
import csv
print("1")
# Read in the csv files.
df1 = dd.read_csv('procedures@seq1.csv')
print("2")
# Merge the csv files.
df = dd.merge(df1, ddf, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print("3")
# # Write the output.
df.head()

1
2
3


,SUBJECT_ID,HADM_ID,ICD9_CODE,TEXT_required
0,9545,158060,34,History of present illness: the patient is an...
1,28600,189217,3613,History of present illness: patient has a two...
2,16994,115444,3601,History of present illness: the patient is a ...
3,13579,186290,3612,History of present illness: this is a 73-year...
4,9090,140708,4399,History of present illness: mr. [**known last...


In [58]:
len(df)

172542

In [57]:
noteevents_190K_proce_top10 = df[df['ICD9_CODE'].isin([9671, 9955, 3893, 9604,  640, 9672, 3521, 3615, 3612,   66])]
len(noteevents_190K_proce_top10)

53437

In [59]:
noteevents_190K_proce_top10 = noteevents_190K_proce_top10[['ICD9_CODE', 'TEXT_required']]

dd.Scalar<series-..., dtype=int64>
dd.Scalar<series-..., dtype=int64>


In [64]:
noteevents_190K_proce_top10.to_csv('noteevents_190K_proce_top10-*.csv', index=False)

['noteevents_190K_proce_top10-0.csv']

In [65]:
noteevents_190K_proce_top10 = pd.read_csv('noteevents_190K_proce_top10-0.csv')

In [66]:
noteevents_190K_proce_top10.head()

,ICD9_CODE,TEXT_required
0,3612,History of present illness: this is a 73-year...
1,9671,History of present illness: the patient is a ...
2,3612,Chief complaint: the patient presents with si...
3,3612,History of present illness: this is a 75-year...
4,9671,History of present illness: this is a 43 -yea...


In [67]:
print(noteevents_190K_proce_top10['ICD9_CODE'].nunique())
print(noteevents_190K_proce_top10['TEXT_required'].nunique())

10
51129


In [118]:
# s = []
# u = []
# for i in noteevents1['TEXT'][:5]:
# #     print(i)
#     if 'history of present illness' in i:
#         a = i.index('history of present illness')
#         b = i.split('history of present illness')[1]
#         print(b)
#         u.append(b)
#         s.append(a)
#     else:
#         b = 'No'
#         print(b)
#         u.append(b)
#         s.append('False')

In [13]:
#output all headings of the text column
import re
finalheadings = []

for i in noteevents['TEXT'][:1000]:
    headings = re.findall(r'(.*?)\s*:', i)
#     headings.mode()[0]
    finalheadings.append(headings)
#     print (headings)

In [14]:
len(finalheadings)

1000

In [15]:
#joining all the lists to a single list
finalcombinedheadings = [item for sublist in finalheadings for item in sublist]

In [16]:
#get frequency of each word in the list
from collections import Counter
counts = Counter(finalcombinedheadings)
print(counts)

Counter({'Disp': 1593, 'Service': 1000, 'Admission Date': 997, 'Date of Birth': 783, 'Past Medical History': 709, 'Allergies': 708, 'Chief Complaint': 707, 'History of Present Illness': 706, 'Social History': 704, 'Attending': 702, 'Physical Exam': 702, 'Major Surgical or Invasive Procedure': 701, 'Brief Hospital Course': 701, 'Discharge Disposition': 701, 'Discharge Diagnosis': 701, 'Discharge Condition': 701, 'Family History': 688, 'Pertinent Results': 685, 'Discharge Instructions': 682, 'Medications on Admission': 679, 'Followup Instructions': 666, 'Discharge Medications': 655, 'IMPRESSION': 648, 'HEENT': 524, 'Facility': 442, '*30 Tablet(s)* Refills': 381, 'CV': 360, 'Completed by': 327, 'T': 317, 'Provider': 305, 'D': 299, 'Neck': 299, 'Dictated By': 298, 'Ext': 287, 'Neuro': 262, 'JOB#': 254, 'HISTORY OF PRESENT ILLNESS': 252, 'PAST MEDICAL HISTORY': 248, 'When': 219, 'ALLERGIES': 217, 'General': 211, 'Lungs': 210, 'HOSPITAL COURSE': 207, 'Date/Time': 206, 'Department': 205, 'Gen

In [8]:
noteevents['DESCRIPTION'].unique()

array(['Report', 'Addendum', 'Nursing Transfer Note', ...,
       'PLACE CATH CAROTID/INOM ART',
       'L US MSK ASPIRATE/INJ GANGLION CYST LEFT',
       'RO HIP NAILING IN OR W/FILMS & FLUORO RIGHT IN O.R.'],
      dtype=object)

In [28]:
pd.isnull(noteevents.iloc[0,9])

True

In [48]:
#tmp
noteeventssubjectuniq = noteevents['SUBJECT_ID'].unique()
len(noteeventssubjectuniq)

46146

In [54]:
# tmp - comparison of unique subject between noteevents and diagnosis
print(len([id7 for id7 in noteeventssubjectuniq if id7 not in diagnosissubjectunique]))
print(len([id8 for id8 in diagnosissubjectunique if id8 not in noteeventssubjectuniq]))

0
374


In [55]:
#tmp - comparison of unique subjects between noteevents and treatment
print(len([id9 for id9 in noteeventssubjectuniq if id9 not in treatmentsubjectunique]))
print(len([id10 for id10 in treatmentsubjectunique if id10 not in noteeventssubjectuniq]))

7070
287


In [56]:
#tmp - comparison of unique subjects between noteevents and procedures
print(len([id11 for id11 in noteeventssubjectuniq if id11 not in proceduressubjectuniq]))
print(len([id12 for id12 in proceduressubjectuniq if id12 not in noteeventssubjectuniq]))

28682
228


# Chartevents dataset

In [2]:
#If the file size is huge, use iterator=True to read the necessary no of rows
import pandas as pd
chartevents = pd.read_csv('CHARTEVENTS.csv', iterator=True)
chartevents.get_chunk(20)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,788,36,165660,241249,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,15.00,15.00,L/min,0,0,NaN,NaN
1,789,36,165660,241249,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,100.00,100.00,NaN,0,0,NaN,NaN
2,790,36,165660,241249,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823,0.37,0.37,NaN,0,0,NaN,NaN
3,791,36,165660,241249,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,6.00,6.00,min,0,0,NaN,NaN
4,792,36,165660,241249,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,2.50,2.50,NaN,0,0,NaN,NaN
5,793,36,165660,241249,224331,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,0.00,0.00,ml/hr,0,0,NaN,NaN
6,794,36,165660,241249,224332,2134-05-12 12:00:00,2134-05-12 14:44:00,17525,3.00,3.00,NaN,0,0,NaN,NaN
7,795,36,165660,241249,224663,2134-05-12 12:00:00,2134-05-12 14:44:00,17525,8.00,8.00,NaN,0,0,NaN,NaN
8,796,36,165660,241249,224665,2134-05-12 12:00:00,2134-05-12 14:44:00,17525,1.11,1.11,NaN,0,0,NaN,NaN
9,797,36,165660,241249,220224,2134-05-12 12:35:00,2134-05-12 12:38:00,20889,58.00,58.00,mmHg,1,0,NaN,NaN


In [8]:
tmp = chartevents.get_chunk(20)
tmp.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
       'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING',
       'ERROR', 'RESULTSTATUS', 'STOPPED'],
      dtype='object')

In [ ]:
# chartevents[:5]

# Diagnosis dataset

In [7]:
import pandas as pd
diagnosis_icd = pd.read_csv('DIAGNOSES_ICD.csv')
# print(len(diagnosis_icd['SUBJECT_ID'].unique()))
# # Concatenating sequence number with icd9 code
# diagnosis_icd['SEQ_ICD9_CODE'] = diagnosis_icd['ICD9_CODE'].map(str) + '--' + diagnosis_icd['SEQ_NUM'].map(str)
print(len(diagnosis_icd))
diagnosis_icd[:10]

651047


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
5,1302,109,172335,6.0,2762
6,1303,109,172335,7.0,7100
7,1304,109,172335,8.0,2767
8,1305,109,172335,9.0,7243
9,1306,109,172335,10.0,45829


In [8]:
diagnosis_icd['ICD9_CODE'].nunique()

6984

In [31]:
diagnosis_icd['SEQ_NUM'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., nan, 31., 32., 33., 34., 35., 36., 37., 38.,
       39.])

In [32]:
diagnosis_icd['SUBJECT_ID'].nunique()

46520

In [33]:
diagnosis_icd['HADM_ID'].nunique()

58976

In [9]:
#only diagnosis with seq no = 1
diagnosis_icd_seq1 = diagnosis_icd[diagnosis_icd['SEQ_NUM'] == 1.]
print(len(diagnosis_icd_seq1))
diagnosis_icd_seq1.head()

58929


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
14,1311,109,173633,1.0,40301
28,1488,112,174105,1.0,53100
33,1493,113,109976,1.0,1915
36,1496,114,178393,1.0,41401


In [10]:
diagnosis_icd_seq1['ICD9_CODE'].nunique()

2789

In [35]:
diagnosis_icd_seq1 = diagnosis_icd_seq1[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
diagnosis_icd_seq1.to_csv('diagnosis@seq1.csv', index=False)

In [69]:
diagnosis_icd_seq1 = pd.read_csv('diagnosis@seq1.csv')
diagnosis_icd_seq1.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,109,172335,40301
1,109,173633,40301
2,112,174105,53100
3,113,109976,1915
4,114,178393,41401


In [70]:
#Top 10 frequent procedures
diagnosis_icd_seq1_top10 = diagnosis_icd_seq1.groupby(['ICD9_CODE'])['ICD9_CODE'].count().reset_index(name='count').sort_values(['count'],ascending=False).head(10)
diagnosis_icd_seq1_top10

,ICD9_CODE,count
2749,V3000,3566
837,41401,3498
2750,V3001,2757
47,0389,2069
826,41071,1751
867,4241,1140
1164,51881,1127
915,431,1030
2753,V3101,998
1101,486,725


In [71]:
diagnosis_icd_seq1_top10['count'].sum()

18661

In [72]:
#percentage of patients with top10 diagnosis
round((18661/58929)*100, 2)

31.67

In [84]:
diagnosis_icd_seq1_top10['ICD9_CODE'].unique()

array(['V3000', '41401', 'V3001', '0389', '41071', '4241', '51881', '431',
       'V3101', '486'], dtype=object)

In [74]:
#Top 10 frequent procedures
diagnosis_icd_seq1_top50 = diagnosis_icd_seq1.groupby(['ICD9_CODE'])['ICD9_CODE'].count().reset_index(name='count').sort_values(['count'],ascending=False).head(50)
diagnosis_icd_seq1_top50

,ICD9_CODE,count
2749,V3000,3566
837,41401,3498
2750,V3001,2757
47,0389,2069
826,41071,1751
867,4241,1140
1164,51881,1127
915,431,1030
2753,V3101,998
1101,486,725


In [75]:
diagnosis_icd_seq1_top50['count'].sum()

31387

In [76]:
#percentage of patients with top50 diagnosis
round((31387/58929)*100, 2)

53.26

In [90]:
diagnosis_icd_seq1_top50['ICD9_CODE'].unique()

array(['V3000', '41401', 'V3001', '0389', '41071', '4241', '51881', '431',
       'V3101', '486', '5070', '4280', '4240', '430', '5849', '41011',
       '41041', '5789', '5770', '1983', '41519', '43411', '43491',
       '42731', '99859', '03842', '85221', '25013', '4373', '42823',
       '99662', '56212', '42833', '49121', '4271', '5712', '51884',
       '29181', 'V3401', '4321', '43310', '99811', '03849', '85220',
       '03811', '042', '4414', '1623', '0380', '5715'], dtype=object)

In [46]:
#Based on unique SUBJECT_ID AND HADM_ID concatenating SEQ_ICD9_CODE into a new column called 'ALL_ICD9_CODES'
diagnosis_icd['ALL_ICD9_CODES'] = diagnosis_icd[['SUBJECT_ID', 'HADM_ID','SEQ_ICD9_CODE']].groupby(['SUBJECT_ID', 'HADM_ID'],
                                                as_index=False)['SEQ_ICD9_CODE'].transform(lambda x: ','.join(x))
diagnosis_icd['ALL_ICD9_CODES'][:10]

0    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
1    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
2    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
3    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
4    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
5    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
6    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
7    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
8    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
9    40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-...
Name: ALL_ICD9_CODES, dtype: object

In [47]:
#Looking at the whole data
diagnosis_icd[:10]

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SEQ_ICD9_CODE,ALL_ICD9_CODES
0,1297,109,172335,1.0,40301,40301--1.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
1,1298,109,172335,2.0,486,486--2.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
2,1299,109,172335,3.0,58281,58281--3.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
3,1300,109,172335,4.0,5855,5855--4.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
4,1301,109,172335,5.0,4254,4254--5.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
5,1302,109,172335,6.0,2762,2762--6.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
6,1303,109,172335,7.0,7100,7100--7.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
7,1304,109,172335,8.0,2767,2767--8.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
8,1305,109,172335,9.0,7243,7243--9.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
9,1306,109,172335,10.0,45829,45829--10.0,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."


In [48]:
#Removing duplicate rows based on SUBJECT_ID, HADM_ID, ALL_ICD9_CODES
diagnosis = diagnosis_icd[['SUBJECT_ID', 'HADM_ID','ALL_ICD9_CODES']].drop_duplicates()
diagnosis

,SUBJECT_ID,HADM_ID,ALL_ICD9_CODES
0,109,172335,"40301--1.0,486--2.0,58281--3.0,5855--4.0,4254-..."
14,109,173633,"40301--1.0,5856--2.0,58381--3.0,7100--4.0,5589..."
28,112,174105,"53100--1.0,41071--2.0,2859--3.0,41401--4.0,725..."
33,113,109976,"1915--1.0,3314--2.0,53081--3.0"
36,114,178393,"41401--1.0,4111--2.0,48283--3.0,2859--4.0,2720..."
42,115,114585,"1940--1.0,1977--2.0,2553--3.0,4240--4.0,5845--..."
60,116,127203,"V3001--1.0,V053--2.0,V290--3.0"
63,117,140784,"5715--1.0,7895--2.0,07054--3.0,2875--4.0,4280-..."
72,117,164853,"570--1.0,07044--2.0,5712--3.0,5849--4.0,5724--..."
94,118,147035,"V3000--1.0,V053--2.0"


In [49]:
diagnosis.to_csv('diagnosis.csv')

In [49]:
#tmp
diagnosis = pd.read_csv('diagnosis.csv')
diagnosissubjectunique = diagnosis['SUBJECT_ID'].unique()
len(diagnosissubjectunique)

46520

In [141]:
diagnosis2 = pd.read_csv('D_ICD_DIAGNOSES.csv')
# len(diagnosis2['ICD9_CODE'].unique())
diagnosis2[:10]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]"
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or histological examination not done"
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or histological examination unknown (at present)"
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli found (in sputum) by microscopy"
5,179,01174,TB pneumothorax-cult dx,"Tuberculous pneumothorax, tubercle bacilli not found (in sputum) by microscopy, but found by bacterial culture"
6,180,01175,TB pneumothorax-histo dx,"Tuberculous pneumothorax, tubercle bacilli not found by bacteriological examination, but tuberculosis confirmed histologically"
7,181,01176,TB pneumothorax-oth test,"Tuberculous pneumothorax, tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]"
8,182,01180,Pulmonary TB NEC-unspec,"Other specified pulmonary tuberculosis, unspecified"
9,183,01181,Pulmonary TB NEC-no exam,"Other specified pulmonary tuberculosis, bacteriological or histological examination not done"


In [142]:
diagnosiscodeanddescrip = pd.merge(diagnosis_icd, diagnosis2, how='left', on='ICD9_CODE')
diagnosiscodeanddescrip = diagnosiscodeanddescrip[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'LONG_TITLE']]
diagnosiscodeanddescrip[:10]

,SUBJECT_ID,HADM_ID,ICD9_CODE,LONG_TITLE
0,109,172335,40301,"Hypertensive chronic kidney disease, malignant, with chronic kidney disease stage V or end stage renal disease"
1,109,172335,486,"Pneumonia, organism unspecified"
2,109,172335,58281,Chronic glomerulonephritis in diseases classified elsewhere
3,109,172335,5855,"Chronic kidney disease, Stage V"
4,109,172335,4254,Other primary cardiomyopathies
5,109,172335,2762,Acidosis
6,109,172335,7100,Systemic lupus erythematosus
7,109,172335,2767,Hyperpotassemia
8,109,172335,7243,Sciatica
9,109,172335,45829,Other iatrogenic hypotension


In [148]:
#Counting na's in all columns
print(len(diagnosiscodeanddescrip))
diagnosiscodeanddescrip.isna().sum()

651047


SUBJECT_ID    0    
HADM_ID       0    
ICD9_CODE     47   
LONG_TITLE    16338
dtype: int64

In [150]:
#dropping values with na's
diagnosiscodeanddescrip = diagnosiscodeanddescrip.dropna(subset = ['ICD9_CODE', 'LONG_TITLE'])
print(len(diagnosiscodeanddescrip))

634709


In [151]:
import os
os.getcwd()

'/home/snuthakk/Capstone'

# Prescription/treatment dataset

In [5]:
prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
prescriptions

/home/snuthakk/.conda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,5.601728e+07,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,5.482972e+07,20mg Tablet,20,mg,1,TAB,PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,5.601698e+07,Check with MD for Dose,1,dose,1,dose,PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,5.601708e+07,2mg Tab,2,mg,1,TAB,PO


In [6]:
prescriptions1 = prescriptions[['SUBJECT_ID', 'HADM_ID', 'DRUG']]
prescriptions1[:10]

,SUBJECT_ID,HADM_ID,DRUG
0,6,107064,Tacrolimus
1,6,107064,Warfarin
2,6,107064,Heparin Sodium
3,6,107064,D5W
4,6,107064,Furosemide
5,6,107064,Warfarin
6,6,107064,Heparin Sodium
7,6,107064,D5W
8,6,107064,Heparin Sodium
9,6,107064,Warfarin


In [7]:
prescriptions1['DRUG'].nunique()

4525

In [103]:
presc_top10 = prescriptions1.groupby(['DRUG'])['DRUG'].count().reset_index(name='count').sort_values(['count'],ascending=False).head(10)
presc_top10

,DRUG,count
3051,Potassium Chloride,192993
1848,Insulin,143465
1036,D5W,142241
1547,Furosemide,133122
124,0.9% Sodium Chloride,130147
2622,NS,129731
2230,Magnesium Sulfate,90427
1904,Iso-Osmotic Dextrose,87005
3398,Sodium Chloride 0.9% Flush,83392
195,Acetaminophen,78768


In [104]:
presc_top10['count'].sum()

1211291

In [155]:
print(len(prescriptions1))
prescriptions1.isna().sum()

4156450


SUBJECT_ID    0
HADM_ID       0
DRUG          0
dtype: int64

In [59]:
#tmp
len(prescriptions['SUBJECT_ID'].unique())

39363

In [38]:
#Creating new column dosage with all the drug, dose and route of administration information
prescriptions['dosage'] = prescriptions['DRUG'].map(str) + ' ' + prescriptions['DOSE_VAL_RX'].map(str) + ' ' + prescriptions['DOSE_UNIT_RX'].map(str) + ' ' + prescriptions['ROUTE'].map(str)
prescriptions[:10]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,dosage
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO,Tacrolimus 2 mg PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO,Warfarin 5 mg PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV,"Heparin Sodium 25,000 UNIT IV"
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV,D5W 250 ml IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO,Furosemide 20 mg PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,56016975.0,Check with MD for Dose,1,dose,1,dose,PO,Warfarin 1 dose PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV,"Heparin Sodium 25,000 UNIT IV"
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV,D5W 250 ml IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV,"Heparin Sodium 25,000 UNIT IV"
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,56017075.0,2mg Tab,2,mg,1,TAB,PO,Warfarin 2 mg PO


In [40]:
#Based on unique SUBJECT_ID AND HADM_ID concatenating 'dosage' into a new column called 'ALL_drugs'
prescriptions['ALL_drugs'] = prescriptions[['SUBJECT_ID', 'HADM_ID','dosage']].groupby(['SUBJECT_ID', 'HADM_ID'],
                                                as_index=False)['dosage'].transform(lambda x: ','.join(x))
prescriptions['ALL_drugs'][:10]

0    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
1    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
2    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
3    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
4    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
5    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
6    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
7    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
8    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
9    Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So...
Name: ALL_drugs, dtype: object

In [51]:
#Removing duplicate rows based on SUBJECT_ID, HADM_ID, ALL_drugs
treatment = prescriptions[['SUBJECT_ID', 'HADM_ID','ALL_drugs']].drop_duplicates()
treatment

,SUBJECT_ID,HADM_ID,ALL_drugs
0,6,107064,"Tacrolimus 2 mg PO,Warfarin 5 mg PO,Heparin So..."
23,13,143045,"D5W 250 ml IV,Heparin Sodium 25000 UNIT IV,Nit..."
71,8,159514,"Send 500mg Vial 1 VIAL IV,NEO*IV*Gentamicin 12..."
75,9,150750,"SW 100 ml IV,Labetalol HCl 10 mg IV,Potassium ..."
107,2,163353,"NEO*IV*Gentamicin 15.5 mg IV,Syringe (Neonatal..."
111,4,185777,"Iso-Osmotic Dextrose 200 ml IV,Insulin 6 UNIT ..."
280,12,112213,"Propofol (Generic) 1000 mg IV DRIP,Levofloxaci..."
359,21,109451,"Vial 1 VIAL IV,Calcium Gluconate 7.5 mEq DIALY..."
435,21,111970,"NS (Mini Bag Plus) 100 ml IV,Levothyroxine Sod..."
487,17,194023,"Sucralfate 1 gm NG,Ketorolac 30 mg IM,LR 1000 ..."


In [53]:
treatment.to_csv('treatments_final.csv')

In [26]:
#tmp
treatment = pd.read_csv('treatments_final.csv')
treatmentsubjectunique = treatment['SUBJECT_ID'].unique()
len(treatmentsubjectunique)

39363

In [41]:
#tmp - Subjects present in treatment but not in diagnosis dataset
treatmentlist = [id1 for id1 in treatmentsubjectunique if id1 not in diagnosissubjectunique]
len(treatmentlist)

0

In [40]:
#tmp - subjects present in diagnosis but not in treatment
len([id4 for id4 in diagnosissubjectunique if id4 not in treatmentsubjectunique])

7157

In [43]:
#tmp - subjects present in treatment but not in procedures
len([id5 for id5 in treatmentsubjectunique if id5 not in proceduressubjectuniq])

21677

# Procedures_MV dataset

In [60]:
#Reading procedures data 
import pandas as pd
procedures = pd.read_csv('PROCEDUREEVENTS_MV.csv')
procedures

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,...,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,...,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
5,384,29070,115071,232563.0,2145-03-13 08:45:00,2145-03-13 08:46:00,225439,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,2,Rewritten,RN,NaN,2145-03-13 10:16:00
6,385,29070,115071,232563.0,2145-03-13 09:00:00,2145-03-13 09:01:00,225439,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
7,386,29070,115071,232563.0,2145-03-14 09:00:00,2145-03-14 09:01:00,225402,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
8,387,29070,115071,232563.0,2145-03-14 10:30:00,2145-03-14 10:31:00,225402,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,2,Rewritten,RN,NaN,2145-03-14 10:30:00
9,388,29070,115071,232563.0,2145-03-15 06:26:00,2145-03-15 06:27:00,225459,1.0,None,NaN,...,Imaging,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN


In [61]:
#tmp
len(procedures['SUBJECT_ID'].unique())

17692

In [6]:
#Based on unique SUBJECT_ID AND HADM_ID concatenating 'ITEMID' into a new column called 'ALL_ITEMIDS'
procedures['ALL_ITEMIDS'] = procedures[['SUBJECT_ID', 'HADM_ID','ITEMID']].groupby(['SUBJECT_ID', 'HADM_ID'],
                                                as_index=False)['ITEMID'].transform(lambda x: ','.join(map(str, x)))  
#Converting 'ITEMID' column to string
procedures[:10]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ALL_ITEMIDS
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,...,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,...,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
5,384,29070,115071,232563.0,2145-03-13 08:45:00,2145-03-13 08:46:00,225439,1.0,None,NaN,...,NaN,Electrolytes,0,0,2,Rewritten,RN,NaN,2145-03-13 10:16:00,"225401,225454,225792,225402,224560,225439,2254..."
6,385,29070,115071,232563.0,2145-03-13 09:00:00,2145-03-13 09:01:00,225439,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
7,386,29070,115071,232563.0,2145-03-14 09:00:00,2145-03-14 09:01:00,225402,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."
8,387,29070,115071,232563.0,2145-03-14 10:30:00,2145-03-14 10:31:00,225402,1.0,None,NaN,...,NaN,Electrolytes,0,0,2,Rewritten,RN,NaN,2145-03-14 10:30:00,"225401,225454,225792,225402,224560,225439,2254..."
9,388,29070,115071,232563.0,2145-03-15 06:26:00,2145-03-15 06:27:00,225459,1.0,None,NaN,...,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN,"225401,225454,225792,225402,224560,225439,2254..."


In [7]:
#Removing duplicate rows based on SUBJECT_ID, HADM_ID, ALL_ITEMIDS
procedure_final = procedures[['SUBJECT_ID', 'HADM_ID','ALL_ITEMIDS']].drop_duplicates()
procedure_final

,SUBJECT_ID,HADM_ID,ALL_ITEMIDS
0,29070,115071,"225401,225454,225792,225402,224560,225439,2254..."
22,31608,152365,"224385,225792,224275,224275,225752,225400,2254..."
41,29059,171662,"224277,225402,225402,223253,223253,225439"
47,5244,190705,"224263,224277,224277,225401,225402,225752,2254..."
55,29200,105865,"227194,224385,224275,224275,224275,221214,2212..."
83,26780,117266,"224277,224275,225439"
86,30354,120396,"224274,224275,224385,225792,226237,224264,2254..."
103,29969,129547,"224264,225402,225402,225441"
107,27063,139787,"224264,225447,225447,225479,225402,225402,2212..."
117,30460,180006,"224277,224277"


In [8]:
procedure_final = procedure_final.to_csv('procedure_final.csv')

In [10]:
#Read procedure_final dataset
procedure_final = pd.read_csv('procedure_final.csv')
procedure_final

,Unnamed: 0,SUBJECT_ID,HADM_ID,ALL_ITEMIDS
0,0,29070,115071,"225401,225454,225792,225402,224560,225439,2254..."
1,22,31608,152365,"224385,225792,224275,224275,225752,225400,2254..."
2,41,29059,171662,"224277,225402,225402,223253,223253,225439"
3,47,5244,190705,"224263,224277,224277,225401,225402,225752,2254..."
4,55,29200,105865,"227194,224385,224275,224275,224275,221214,2212..."
5,83,26780,117266,"224277,224275,225439"
6,86,30354,120396,"224274,224275,224385,225792,226237,224264,2254..."
7,103,29969,129547,"224264,225402,225402,225441"
8,107,27063,139787,"224264,225447,225447,225479,225402,225402,2212..."
9,117,30460,180006,"224277,224277"


In [35]:
#tmp
proceduressubjectuniq = procedure_final['SUBJECT_ID'].unique()
proceduressubjectuniq
len(proceduressubjectuniq)

17692

In [45]:
#tmp - subjects present in treatment but not in procedures
len([id6 for id6 in diagnosissubjectunique if id6 not in proceduressubjectuniq])

28828

In [38]:
#tmp -  subject_ids present in procedures but not in treatment
len([id2 for id2 in proceduressubjectuniq if id2 not in treatmentsubjectunique])

6

In [46]:
# tmp - Subject_ids present in procedures but not in diagnosis
len([id3 for id3 in proceduressubjectuniq if id3 not in diagnosissubjectunique])

0

# Procedures_ICD dataset

In [2]:
procedures_icd = pd.read_csv('PROCEDURES_ICD.csv')
procedures_icd[:10]

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
5,949,55357,119355,3,3893
6,950,9545,158060,1,34
7,951,28600,189217,1,3613
8,952,28600,189217,2,3615
9,953,28600,189217,3,3961


In [3]:
print(len(procedures_icd))
procedures_icd['ICD9_CODE'].nunique()

240095


2009

In [4]:
#only procedures with seq no = 1
procedures_icd_seq1 = procedures_icd[procedures_icd['SEQ_NUM'] == 1.]
print(len(procedures_icd_seq1))
procedures_icd_seq1.head()

52243


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
1,945,2592,130856,1,9671
3,947,55357,119355,1,9672
6,950,9545,158060,1,34
7,951,28600,189217,1,3613
10,954,16994,115444,1,3601


In [6]:
procedures_icd_seq1['ICD9_CODE'].nunique()

1285

In [8]:
procedures_icd_seq1 = procedures_icd_seq1[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
procedures_icd_seq1.head()
# procedures_icd_seq1.to_csv('procedures@seq1.csv', index=False)

,SUBJECT_ID,HADM_ID,ICD9_CODE
1,2592,130856,9671
3,55357,119355,9672
6,9545,158060,34
7,28600,189217,3613
10,16994,115444,3601


In [19]:
#Top 10 frequent procedures
procedures_icd_seq1_top10 = procedures_icd_seq1.groupby(['ICD9_CODE'])['ICD9_CODE'].count().reset_index(name='count').sort_values(['count'],ascending=False).head(10)
procedures_icd_seq1_top10

,ICD9_CODE,count
1225,9671,3349
1273,9955,2667
464,3893,1947
1206,9604,1866
161,640,1707
1226,9672,1620
347,3521,1358
378,3615,1178
375,3612,1029
19,66,1017


In [41]:
procedures_icd_seq1_top10['count'].sum()

17738

In [45]:
#percentage of patients with top10 diagnosis
round((17738/52243)*100, 2)

33.95

In [24]:
procedurestop10 = procedures_icd_seq1_top10['ICD9_CODE'].unique()
procedurestop10

array([9671, 9955, 3893, 9604,  640, 9672, 3521, 3615, 3612,   66])

In [22]:
procedurestop10 = [9671, 9955, 3893, 9604, 640, 9672, 3521, 3615, 3612, 66]

In [20]:
#Top 50 frequent procedures
procedures_icd_seq1_top50 = procedures_icd_seq1.groupby(['ICD9_CODE'])['ICD9_CODE'].count().reset_index(name='count').sort_values(['count'],ascending=False).head(50)
procedures_icd_seq1_top50

,ICD9_CODE,count
1225,9671,3349
1273,9955,2667
464,3893,1947
1206,9604,1866
161,640,1707
1226,9672,1620
347,3521,1358
378,3615,1178
375,3612,1029
19,66,1017


In [23]:
procedurestop50 = procedures_icd_seq1_top50['ICD9_CODE'].unique()
procedurestop50

array([9671, 9955, 3893, 9604,  640, 9672, 3521, 3615, 3612,   66, 9390,
        311, 4513, 3613, 3601, 3995, 9983,  966,  159, 3522, 9904,  331,
        131, 9915, 3950, 3512, 3611, 3761, 3972, 5491, 4443, 3723,  387,
       3614, 3734, 3897, 3891, 5059, 8841, 3523, 8622, 7935, 3979, 3491,
       3722, 9910, 4516,   22, 3929, 4562])

In [46]:
procedures_icd_seq1_top50['count'].sum()

34427

In [47]:
#percentage of patients with top50 diagnosis
round((34427/52243)*100, 2)

65.9

In [7]:
procedures_icd = procedures_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
procedures_icd[:10]

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,62641,154460,3404
1,2592,130856,9671
2,2592,130856,3893
3,55357,119355,9672
4,55357,119355,331
5,55357,119355,3893
6,9545,158060,34
7,28600,189217,3613
8,28600,189217,3615
9,28600,189217,3961


In [161]:
procedures_icd.isna().sum()

SUBJECT_ID    0
HADM_ID       0
ICD9_CODE     0
dtype: int64

In [66]:
procicduniq = procedures_icd['SUBJECT_ID'].unique()
len(procedures_icd['SUBJECT_ID'].unique())

42214

In [68]:
#tmp - compare subjects between procedures_icd and noteevents
print(len([id13 for id13 in procicduniq if id13 not in noteeventssubjectuniq]))
print(len([id14 for id14 in noteeventssubjectuniq if id14 not in procicduniq]))

328
4260


In [69]:
#tmp - compare unique subjects between procedures_icd and treatment events
print(len([id13 for id13 in procicduniq if id13 not in treatmentsubjectunique]))
print(len([id14 for id14 in treatmentsubjectunique if id14 not in procicduniq]))

6445
3594


In [ ]:
d_items = pd.read_csv('D_ITEMS.csv')
d_items

In [ ]:
d_items['']

# Merging datasets

In [162]:
import time
start_time = time.time()
diagproce = pd.merge(procedures_icd, diagnosiscodeanddescrip, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print(len(diagproce))
print(diagproce[:10])
print("--- %s seconds ---" % (time.time() - start_time))

3085432
   SUBJECT_ID  HADM_ID  ICD9_CODE_x ICD9_CODE_y  \
0  62641       154460   3404         1623         
1  62641       154460   3404         9971         
2  62641       154460   3404         42731        
3  62641       154460   3404         99679        
4  62641       154460   3404         5121         
5  62641       154460   3404         99791        
6  62641       154460   3404         45182        
7  62641       154460   3404         V1051        
8  62641       154460   326          1623         
9  62641       154460   326          9971         

                                                                                     LONG_TITLE  
0  Malignant neoplasm of upper lobe, bronchus or lung                                            
1  Cardiac complications, not elsewhere classified                                               
2  Atrial fibrillation                                                                           
3  Other complications due to other in

In [163]:
diagproce.columns = ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE_PROCEDURES', 'ICD9_CODE_DIAGNOSIS', 'LONG_TITLE']
diagproce[:10]

,SUBJECT_ID,HADM_ID,ICD9_CODE_PROCEDURES,ICD9_CODE_DIAGNOSIS,LONG_TITLE
0,62641,154460,3404,1623,"Malignant neoplasm of upper lobe, bronchus or lung"
1,62641,154460,3404,9971,"Cardiac complications, not elsewhere classified"
2,62641,154460,3404,42731,Atrial fibrillation
3,62641,154460,3404,99679,"Other complications due to other internal prosthetic device, implant, and graft"
4,62641,154460,3404,5121,Iatrogenic pneumothorax
5,62641,154460,3404,99791,"Complications affecting other specified body systems, not elsewhere classified, hypertension"
6,62641,154460,3404,45182,Phlebitis and thrombophlebitis of superficial veins of upper extremities
7,62641,154460,3404,V1051,Personal history of malignant neoplasm of bladder
8,62641,154460,326,1623,"Malignant neoplasm of upper lobe, bronchus or lung"
9,62641,154460,326,9971,"Cardiac complications, not elsewhere classified"


In [164]:
diagproce.isna().sum()

SUBJECT_ID              0
HADM_ID                 0
ICD9_CODE_PROCEDURES    0
ICD9_CODE_DIAGNOSIS     0
LONG_TITLE              0
dtype: int64

In [166]:
diagproce.to_csv('diagproce.csv')

In [167]:
import time
start_time = time.time()
merge2 = pd.merge(diagproce, prescriptions1, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print(len(merge2))
merge2[:10]
print("--- %s seconds ---" % (time.time() - start_time))

461500148
--- 62.347370624542236 seconds ---


In [170]:
merge2[:10]

,SUBJECT_ID,HADM_ID,ICD9_CODE_PROCEDURES,ICD9_CODE_DIAGNOSIS,LONG_TITLE,DRUG
0,11705,165730,966,V3101,"Twin birth, mate liveborn, born in hospital, delivered by cesarean section",NEO*PO*Ferrous Sulfate Elixir
1,11705,165730,966,7783,Other hypothermia of newborn,NEO*PO*Ferrous Sulfate Elixir
2,11705,165730,966,7742,Neonatal jaundice associated with preterm delivery,NEO*PO*Ferrous Sulfate Elixir
3,11705,165730,966,76406,"""Light-for-dates"" without mention of fetal malnutrition, 1,500- 1,749 grams",NEO*PO*Ferrous Sulfate Elixir
4,11705,165730,966,V053,Need for prophylactic vaccination and inoculation against viral hepatitis,NEO*PO*Ferrous Sulfate Elixir
5,11705,165730,966,V293,Observation for suspected genetic or metabolic condition,NEO*PO*Ferrous Sulfate Elixir
6,11705,165730,9983,V3101,"Twin birth, mate liveborn, born in hospital, delivered by cesarean section",NEO*PO*Ferrous Sulfate Elixir
7,11705,165730,9983,7783,Other hypothermia of newborn,NEO*PO*Ferrous Sulfate Elixir
8,11705,165730,9983,7742,Neonatal jaundice associated with preterm delivery,NEO*PO*Ferrous Sulfate Elixir
9,11705,165730,9983,76406,"""Light-for-dates"" without mention of fetal malnutrition, 1,500- 1,749 grams",NEO*PO*Ferrous Sulfate Elixir


In [169]:
merge2.to_csv('diagprocedrug.csv')

KeyboardInterrupt: 

In [ ]:
# df1 = pd.read_csv('diagprocedrug.csv')
# df1[:5]

In [180]:
noteevents2 = pd.read_csv('noteevents2.csv')
noteevents2.head()

Unnamed: 0  SUBJECT_ID   HADM_ID  \
0  1           13702       107527.0   
1  2           13702       167118.0   
2  3           13702       196489.0   
3  4           26880       135453.0   
4  5           53181       170490.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [181]:
noteevents2 = noteevents2[['SUBJECT_ID', 'HADM_ID', 'TEXT_required']]

### http://docs.dask.org/en/latest/dataframe-api.html
http://docs.dask.org/en/latest/dataframe-groupby.html

In [185]:
import time
start_time = time.time()
merge3 = pd.merge(merge2, noteevents2, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print(len(merge3))
merge3[:10]
print("--- %s seconds ---" % (time.time() - start_time))

MemoryError: 

In [182]:
#converting pandas dataframe 'noteevents1' to dask dataframe
import dask.dataframe as dd
# ddf = dd.read_csv('diagprocedrug.csv')
ddf = dd.from_pandas(noteevents2, npartitions=1)
ddf.head(6)
# len(ddf)

SUBJECT_ID   HADM_ID  \
0  13702       107527.0   
1  13702       167118.0   
2  13702       196489.0   
3  26880       135453.0   
4  53181       170490.0   
5  20646       134727.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [186]:
import dask.multiprocessing
import csv
print("1")
# Read in the csv files.
df1 = dd.read_csv('diagprocedrug.csv')
# print("2")
# df2 = dd.read_csv('NOTEEVENTS.csv', sep='\n', quoting=csv.QUOTE_NONE, error_bad_lines=False)
# df3 = df2.compute(get=dask.multiprocessing.get)     # convert to pandas
# pint(len(df2))
# print(df2.columns)
print("3")
# Merge the csv files.
# dd.merge(a, pandas_df)  # fast  #http://docs.dask.org/en/latest/dataframe-performance.html
df = dd.merge(df1, ddf, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print("4")
# # Write the output.
df.head()

1
3
4


,Unnamed: 0,SUBJECT_ID,HADM_ID,ICD9_CODE_PROCEDURES,ICD9_CODE_DIAGNOSIS,LONG_TITLE,DRUG,TEXT_required
0,0,11705,165730,966,V3101,"Twin birth, mate liveborn, born in hospital, delivered by cesarean section",NEO*PO*Ferrous Sulfate Elixir,"History of present illness: 1,700 gram, 35 and 2/7 weeks\ngestation, twin female; small for gestational age, admitted\nfor temperature instability. this twin, number one, born by\ncesarean section to a 31 year old, gravida i, para 0 mother.\npregnancy complicated by insulin dependent diabetes mellitus\nand subchorionic bleed at 14 weeks. infant was admitted to\nthe newborn nursery but had difficulty maintaining\ntemperature and with p.o. feeding. apgars were eight at one\nminute and nine at five minutes. no sepsis risk factors. infant\nwas transferred to neonatal intensive care unit.\n\nprenatal screens: 0 positive, antibody negative, hepatitis b\nsurface antigen negative. rpr nonreactive. rubella immune.\ngbs unknown.\n\nphysical examination: birth weight 1,700 grams (less than\n10th percentile); length 48.5 cm (75th percentile); head\ncircumference 30 cm (10 to 25 percentile). pink, well\nperfused, no rash or petechiae. anterior fontanel flat.\nclear breath sounds bilaterally. normal s1, s2, no murmur,\npulses normal. abdomen soft, normal female genitalia, moving\nall extremities. normal tone and reflexes. hips stable.\nspine infact. palate intact.\n\nhospital course: respiratory: infant has remained in room\nair throughout this hospitalization with respiratory rates 30\nto 50's. infant has not had any apnea or bradycardia during\nthis hospitalization.\n\ncardiovascular: infant has remained hemodynamically stable\nthis hospitalization. no murmur. heart rate 120 to 150.\nmean blood pressures have been 46 to 56.\n\nfluids, electrolytes and nutrition: infant has been taking\np.o. feedings throughout this hospitalization without\ndifficulty. she is currently taking a minimum of 100 cc per\nkg per day of neosure 24 calories per ounce. infant was\ninitially started on enfamil 20 calories per ounce or breast\nmilk 20 calories per ounce. infant has tolerated feeding\nadvancement without difficulties. glucoses have been stable.\nthe most recent weight is 1,730 grams; head circumference 30\ncm; length 44 cm.\n\ngastrointestinal: the patient was started under single\nphototherapy on day of life three. maximum bilirubin of 9.2\nwith a direct of 0.3. phototherapy was discontinued on day\nof life six and a rebound bilirubin on day of discharge was\n6.6/0.4.\n\nhematology: blood type is 0 positive. direct coombs\nnegative. infant has not received any blood transfusions\nthis hospitalization. infant has not had a cbc and has had\nno issues with infection.\n\ninfectious disease: no issues.\n\nneurology: normal neurologic examination.\n\nsensory: hearing screening was performed with automated\nauditory brain stem responses. results: passed bilaterally.\npassed car seat positioning study.\n\n\npsychosocial: social work was involved with the family. the\ncontact social worker can be reached at [**telephone/fax (1) 8717**]. the\nother twin was discharged home from the newborn nursery.\n\ncondition on discharge: stable on room air.\n\ndischarge disposition: home with parents.\n\nname of pediatrician: dr. [**first name4 (namepattern1) **] [**last name (namepattern1) 94402**]- [**location (un) 8117**], nh. phone number\n[**telephone/fax (1) 94403**].\n\nfeedings at discharge: neosure 24 calories per ounce is\nrecommended until six to nine months corrected age.\n\nmedications: none.\n\ncar seat position screening:\n\nstate newborn screen: sent on [**11-21**]; results are\npending.\n\nimmunizations: infant received hepatitis b vaccine on\n[**2165-11-25**].\n\ndischarge diagnoses:\n1. prematurity at 35 and 2/7 weeks, twin #1.\n2. small for gestational age.\n3. status post indirect hyperbilirubinemia.\n\n\n\n\n\n dr.[**last name (stitle) 37692**],[**first name3 (lf) 37693**] 50-454\n\ndictated by:[**last name (namepatte

In [188]:
df.isna().sum()

Dask Series Structure:
npartitions=1
DRUG          int64
Unnamed: 0    ...  
dtype: int64
Dask Name: dataframe-sum-agg, 1964 tasks

In [187]:
len(df)

/usr/local/anaconda3/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


1880404305

In [189]:
len(df1)

/usr/local/anaconda3/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


233224071

In [190]:
dask.set_options(get=dask.local.get_sync)

In [192]:
import time
start_time = time.time()
df.to_csv('final-*.csv', index=False)
print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/anaconda3/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


KeyboardInterrupt: 